Imports

In [ ]:
import pandas as pd
import math
import folium
from folium import Marker

Loading the data

In [ ]:
population_data = pd.read_csv('2019_Census_US_Population_Data_By_State_Lat_Long.csv')
vaccination_data = pd.read_csv('USA_VaccinationData.csv')

#run the commented code below to get latest data online rather than using the preloaded csv file
#vaccination_data = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')

In [ ]:
#getting dates with non NAN values
valid_dates = vaccination_data.loc[(vaccination_data.people_vaccinated.isna() == False)]['date'] #to get dates that have dont have nan values

Getting date from user that is valid

In [ ]:
yr = input("Enter the year as YYYY")
m = input("Enter month as MM")
options = []
dt = yr + '-' + m
for i in valid_dates:
    if dt in i[:7]:
        if i[8:] in options:
            pass
        else:
            options.append(i[8:])
d = input(f"Choose and enter the date from the following as DD {options}")
dt = dt + '-' + d

In [ ]:
#Creates a new filtered table with the data of date chosen by the user
filtered = vaccination_data.loc[(vaccination_data.people_vaccinated.isna() == False)&(vaccination_data.date == dt)][['location','people_vaccinated']]

Merges the population and vaccination tables together

In [ ]:
merged = pd.merge(population_data,filtered,left_on='STATE',right_on='location').drop(columns='location')
merged['percentage_vaccinated'] = merged['people_vaccinated'] / merged['POPESTIMATE2019']

Calculates the percentage vaccinated

In [ ]:
total_per = merged['people_vaccinated'].sum() / merged['POPESTIMATE2019'].sum()
per = round(total_per*100,2)
print(f"On {dt} {per}% of US was vaccinated")

Visualising Map using Marker

In [ ]:
vaccination_map = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=4)

for idx,row in merged.iterrows():
    Marker([row['lat'],row['long']],tooltip=str(round(row['percentage_vaccinated']*100,2))+'%',popup=row['STATE']).add_to(vaccination_map)

display(vaccination_map)